<a href="https://colab.research.google.com/github/pollyjuice74/5G-Decoder/blob/main/LTD_model_reg_LDPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/pollyjuice74/5G-Decoder
!pip install sionna

Cloning into '5G-Decoder'...
remote: Enumerating objects: 1467, done.
remote: Counting objects: 100% (1467/1467), done.
remote: Compressing objects: 100% (527/527), done.
remote: Total 1467 (delta 929), reused 1462 (delta 926), pack-reused 0 (from 0)
Receiving objects: 100% (1467/1467), 1.65 MiB | 6.94 MiB/s, done.
Resolving deltas: 100% (929/929), done.
INFO: pip is looking at multiple versions of ipywidgets to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of ipywidgets to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.6/271.6 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.3/384.3 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━

In [ ]:

import tensorflow as tf
import random
import numpy as np
import time
from scipy.sparse import issparse, csr_matrix

from sionna.fec.utils import generate_reg_ldpc, load_parity_check_examples, LinearEncoder, gm2pcm
from sionna.utils.plotting import PlotBER
from sionna.fec.ldpc import LDPCBPDecoder

import os
# os.chdir('../..')
if os.path.exists('5G-Decoder'):
  os.rename('5G-Decoder', '5G_Decoder')
os.chdir('5G_Decoder/adv_nn')

from dataset import *
from attention import *
from channel import *
from args import *
from model_functs import *
from models import *

In [ ]:
print("Loading LDPC code")
pcm, k, n, coderate = generate_reg_ldpc(v=3,
                                        c=6,
                                        n=10,
                                        allow_flex_len=True,
                                        verbose=True)

# pcm = tf.cast(pcm, dtype=tf.int32)
encoder = LinearEncoder(pcm, is_pcm=True, dtype=tf.int32)

batch_size = 1  # For multiple codewords
b = tf.random.uniform((batch_size, k), minval=0, maxval=2, dtype=tf.int32)
c = encoder(b)

pcm @ tf.reshape(c, (-1, batch_size)) % 2

In [30]:
# for e2e model
from sionna.utils import BinarySource, ebnodb2no
from sionna.mapping import Mapper, Demapper
from sionna.channel import AWGN
# from sionna.fec.ldpc import LDPC5GDecoder, LDPC5GEncoder
from tensorflow.keras.layers import Layer, Dense, Dropout
import matplotlib.pyplot as plt


class Args():
    def __init__(self, model_type, code_type='LDPC', n_look_up=121, k_look_up=80, n=400, k=200,
                       n_rings=2, ls_active=True, split_diff=True, sigma=0.1,
                       t_layers=1, d_model=128, heads=8, lr=5e-4,
                       batch_size=160, batch_size_eval = 150,
                       eval_train_iter=5, save_weights_iter=100,
                       ebno_db_eval=2.5,
                       ebno_db_min=0., ebno_db_max=4., ebno_db_stepsize=0.25,
                       traindata_len=500, testdata_len=250, epochs=1000):
        assert model_type in ['gen', 'dis'], "Type must be: 'gen', Generator or 'dis', Discriminator."
        assert code_type in ['POLAR', 'BCH', 'CCSDS', 'LDPC', 'MACKAY', 'LDPC5G', 'POLAR5G'], "Invalid linear code type."


        # model data
        self.model_type = model_type

        self.split_diff = split_diff
        self.n_rings = n_rings # ring connectivity of mask
        self.sigma = sigma
        self.t_layers = t_layers
        self.ls_active = ls_active

        self.d_model = d_model
        self.heads = heads

        # training data
        self.lr = lr
        self.batch_size = batch_size
        self.traindata_len = traindata_len
        self.testdata_len = testdata_len
        self.epochs = epochs

        self.ebno_db_min = ebno_db_min
        self.ebno_db_max = ebno_db_max
        self.ebno_db_stepsize = ebno_db_stepsize

        self.ebno_db_eval = ebno_db_eval
        self.eval_train_iter = eval_train_iter
        self.save_weights_iter = save_weights_iter
        self.batch_size_eval = batch_size_eval

        # code data
        self.code_type = code_type
        self.code = self.get_code(n_look_up, k_look_up) # n,k look up values in Get_Generator_and_Parity

        # if self.code_type not in ['LDPC5G', 'POLAR5G']:
        #     self.n, self.m, self.k = self.code.n, self.code.m, self.code.k
        # else:
        #     self.n, self.m, self.k = n, n-k, k

        # self.n_steps = self.m + 5  # Number of diffusion steps

    def get_code(self, n_look_up, k_look_up):
        code = type('Code', (), {})() # class Code, no base class, no attributes/methods, () instantiate object
        # code.n_look_up, code.k_look_up = n_look_up, k_look_up
        # code.code_type = self.code_type

        # if self.code_type not in ['LDPC5G', 'POLAR5G']:
        #     G, H = Get_Generator_and_Parity(code)
        #     code.G, code.H = tf.convert_to_tensor(G), csr_matrix( tf.convert_to_tensor(H) )

        #     code.m, code.n = code.H.shape
        #     code.k = code.n - code.m

        return code


class MHAttention(Layer):
    def __init__(self, dims, heads, mask_length, linear=False, dropout=0.01):
        super().__init__()
        assert (dims % heads) == 0, 'dimension must be divisible by the number of heads'
        self.linear = linear
        self.dims = dims
        self.heads = heads
        self.dim_head = dims // heads

        if linear:
            self.k_proj = self.get_k_proj(mask_length) # n+m
            self.proj_k = None
            self.proj_v = None

        self.to_q, self.to_k, self.to_v = [ Dense(self.dims, use_bias=False) for _ in range(3) ]
        self.to_out = Dense(dims)
        self.dropout = Dropout(dropout) # to d-dimentional embeddings

    def build(self, input_shape):
        # Creates shape (n,k_proj) proj matrices for key and
        n_value = input_shape[1]
        if self.linear:
            self.proj_k = self.add_weight("proj_k", shape=[n_value, self.k_proj], initializer=GlorotUniform())
            self.proj_v = self.add_weight("proj_v", shape=[n_value, self.k_proj], initializer=GlorotUniform())

    def call(self, x, mask=None):
        out_att = self.lin_attention(x, mask) if self.linear else self.attention(x, mask)
        return out_att

    def get_k_proj(self, mask_length):
        # gets dimention for linear tranformer vector projection
        for k_proj in range(mask_length // 2, 0, -1): # starts at half the mask length TO 0
            if mask_length % k_proj == 0:
                return tf.cast(k_proj, tf.int32)

    def lin_attention(self, x, mask): # O(n)
        shape = tf.shape(x) # (b, n, d)
        b = tf.cast(shape[0], tf.int32)
        n = tf.cast(shape[1], tf.int32)

        assert x.shape[-1] is not None, "The last dimension of x is undefined."

        query, key, val = self.to_q(x), self.to_k(x), self.to_v(x)

        # Project key and val into k-dimentional space
        key = tf.einsum('bnd,nk->bkd', key, self.proj_k)
        val = tf.einsum('bnd,nk->bkd', val, self.proj_v)

        # Reshape splitting for heads
        query = tf.reshape(query, (b, n, self.heads, self.dim_head))
        key = tf.reshape(key, (b, self.k_proj, self.heads, self.dim_head))
        val = tf.reshape(val, (b, self.k_proj, self.heads, self.dim_head))
        query, key, val = [ tf.transpose(x, [0, 2, 1, 3]) for x in [query, key, val] ]

        # Low-rank mask (n,k_proj)
        mask = tf.expand_dims(mask, axis=-1)
        mask = tf.image.resize(mask, [n, self.k_proj], method='nearest')
        mask = tf.reshape(mask, (1, 1, n, self.k_proj))

        # Main attn logic: sftmx( q@k / d**0.5 ) @ v
        scores = tf.einsum('bhnd,bhkd->bhnk', query, key) / (tf.sqrt( tf.cast(self.dim_head, dtype=tf.float32) ))
        scores += (mask * -1e9) if mask is not None else 0.
        attn = tf.nn.softmax(scores, axis=-1) # (b,h,n,k_proj)
        attn = self.dropout(attn)
        out = tf.einsum('bhnk,bhkd->bhnd', attn, val)

        # Reshape and pass through out layer
        out = tf.transpose(out, [0, 2, 1, 3])
        out = tf.reshape(out, (b, n, -1))
        return self.to_out(out)

    def attention(self, x, mask): # O(n^2)
        shape = tf.shape(x)
        b = shape[0]
        n = shape[1]
        x = x[:, :, tf.newaxis] # (b,n,1)

        query, key, val = self.to_q(x), self.to_k(x), self.to_v(x) # (b, n, d)
        query, key, val = [ tf.reshape(x, (b, n, self.heads, self.dim_head)) for x in [query, key, val] ]
        query, key, val = [ tf.cast( tf.transpose(x, [0, 2, 1, 3]), tf.float32 )
                                                                            for x in [query, key, val] ]

        scores = tf.einsum('bhqd,bhkd->bhqk', query, key) / (tf.sqrt( tf.cast(self.dim_head, tf.float32) ))
        scores += (mask * -1e9) if mask is not None else 0. # apply mask non-edge connections
        attn = tf.nn.softmax(scores, axis=-1) #-1
        attn = self.dropout(attn)
        out = tf.einsum('bhqk,bhkd->bhqd', attn, val)

        out = tf.transpose(out, [0, 2, 1, 3])
        out = tf.reshape(out, (b, n, -1))
        return self.to_out(out)


# class Decoder( TransformerDiffusion ):
#     def __init__(self, args):
#         super().__init__(args)
#         self.transformer =

#     # 'test' function
#     def call(self, r_t):
#         i = tf.constant(0)  # Initialize loop counter

#         def condition(r_t, i):
#             # Loop while i < self.m and syndrome sum is not zero
#             return tf.logical_and(i < 5, tf.reduce_sum(self.get_syndrome(r_t)) != 0) # CHANGE 5 TO SELF.M

#         def body(r_t, i):
#             # Perform reverse or split diffusion
#             r_t = tf.cond(
#                 tf.logical_not(self.split_diff),
#                 lambda: self.split_rdiff_call(r_t),
#                 lambda: self.rev_diff_call(r_t),
#             )
#             return r_t, tf.add(i, 1)

#         # Run tf.while_loop with the loop variables
#         llr_hat, _ = tf.while_loop(
#             condition,
#             body,
#             loop_vars=[r_t, i],
#             maximum_iterations=self.m,
#             shape_invariants=[tf.TensorShape([self.batch_size, self.n]), i.get_shape()]
#         )

#         # llr_hat, _ = self.tran_call(r_t)
#         tf.print("llr_hat", llr_hat)

#         return llr_hat

#     # Refines recieved codeword r at time t
#     def rev_diff_call(self, r_t):
#         tf.print("Rev def call with line-search...")

#         # Transformer error prediction
#         z_hat_crude, t = self.tran_call(r_t) # (b,n)
#         r_t1 = r_t - z_hat_crude*self.get_sigma(t)[:, tf.newaxis] # (b,n)
#         # tf.print(r_t1)

#         # # Refined estimate of the codeword for the ls diffusion step
#         # r_t1, z_hat = self.line_search(r_t, sigma, err_hat) if self.ls_active else 1.
#         # tf.print("After linesearch: ", r_t1)

#         print("r_t1", r_t1.shape, r_t1.dtype)
#         return r_t1 # r at t-1, both (b,n)

#     def split_rdiff_call(self, r_t):
#         tf.print("Rev diff call with split diffusion...")
#         # First half-step condition subproblem
#         z_hat_crude, t = self.tran_call(r_t)
#         # tf.print("fc input: ", (z_hat_crude * self.get_sigma(t)[:, tf.newaxis]))
#         r_t_half = r_t - 0.5 * self.fc( z_hat_crude * self.get_sigma(t)[:, tf.newaxis] )
#         # tf.print("r_t_half", r_t_half)

#         # Full-step diffusion subproblem
#         r_t1 = r_t_half + tf.random.normal(r_t_half.shape) * tf.sqrt(self.get_sigma(t)[:, tf.newaxis])

#         # Second half-step condition subproblem
#         z_hat_crude_half, t = self.tran_call(r_t1)  # Reuse the second `tran_call`
#         r_t1 = r_t1 - 0.5 * self.fc(z_hat_crude_half * self.get_sigma(t)[:, tf.newaxis])
#         print("r_t1", r_t1.shape, r_t1.dtype)
#         return r_t1  # r at t-1, both (b,n)


from tensorflow.keras.layers import MultiHeadAttention, Dense, LayerNormalization, Dropout

class TransformerDecoderBlock(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, dropout_rate=0.1):
        super(TransformerDecoderBlock, self).__init__()
        self.mha = MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.ffn = tf.keras.Sequential([
            Dense(dff, activation='relu'),
            Dense(d_model)
        ])

        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)

        self.dropout1 = Dropout(dropout_rate)
        self.dropout2 = Dropout(dropout_rate)

    def call(self, x, mask, training):
        # Multi-Head Attention
        attn_output = self.mha(x, x, attention_mask=mask)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)  # Add & Normalize

        # Feedforward Network
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)  # Add & Normalize
        return out2


class Decoder( Layer ):
    def __init__(self, args):
        super().__init__()
        code = args.code
        self.pcm = tf.cast(code.H, dtype=tf.int32)

        # shapes
        self._m, self._n = self.pcm.shape
        self._k = self._n - self._m
        self.dims = args.d_model
        self.batch_size = args.batch_size

        # layers
        self.encoder_blocks = [
            TransformerDecoderBlock(
                d_model=args.d_model,
                num_heads=args.heads,
                dff=args.d_model * 4,
                dropout_rate=0.1,
            )
            for _ in range(args.t_layers)
        ]
        self.forward_channel = Dense(1)
        self.to_n = Dense(self._n)

        # mask
        self.mask = self.create_mask(self.pcm)
        # for matrix, title in zip([self.pcm, self.mask], ["PCM Matrix", "Mask Matrix"]):
        #     plt.imshow(matrix, cmap='viridis'); plt.colorbar(); plt.title(title); plt.show()
        # print("mask, pcm: ", self.mask, self.pcm)

    def create_mask(self, H):
        # Initialize diagonal identity mask
        mask = tf.eye(2 * self._n - self._k, dtype=tf.float32)

        # Get indices where H == 1
        indices = tf.where(H == 1)  # Returns (row, col) pairs where H is 1
        check_nodes, variable_nodes = indices[:, 0], indices[:, 1]

        # Step 1: Update check node to variable node connections
        mask = tf.tensor_scatter_nd_update(mask,
                                          tf.stack([n + check_nodes, variable_nodes], axis=1),
                                          tf.ones_like(check_nodes, dtype=tf.float32))
        mask = tf.tensor_scatter_nd_update(mask,
                                          tf.stack([variable_nodes, n + check_nodes], axis=1),
                                          tf.ones_like(check_nodes, dtype=tf.float32))

        # Step 2: Update variable node connections
        for cn in tf.unique(check_nodes)[0]:  # Iterate over unique check nodes
            related_vns = tf.boolean_mask(variable_nodes, check_nodes == cn)
            indices = tf.stack(tf.meshgrid(related_vns, related_vns), axis=-1)
            indices = tf.reshape(indices, [-1, 2])  # Flatten indices
            mask = tf.tensor_scatter_nd_update(mask, indices, tf.ones_like(indices[:, 0], dtype=tf.float32))

        return mask

    def get_syndrome(self, llr_vector):
        """ Calculate syndrome (pcm @ r = 0) if r is correct in binary """
        llr_vector = tf.reshape(llr_vector, (self._n, -1)) # (n,b)
        bin_vector = tf.cast(llr_to_bin(llr_vector), dtype=tf.int32)

        return tf.cast( (self.pcm @ bin_vector) % 2, dtype=tf.float32) # (m,n)@(n,b)->(m,b)

    def call(self, x_nodes, training=False):
        tf.print("DECODER CALL")
        # Pass through each encoder block
        for block in self.encoder_blocks:
            x_nodes = block(x_nodes,
                            mask=self.mask,
                            training=training)
            tf.print("x_nodes", x_nodes)
        x_nodes = tf.squeeze( self.forward_channel(x_nodes), axis=-1 ) # (b, n+m, hidden_dims)->(b, n+m)
        tf.print("x_nodes", x_nodes, x_nodes.shape)
        llr_hat = self.to_n(x_nodes) # (b, n+m)->(b,n)
        tf.print("Decoded output (llr_hat):", llr_hat)
        return llr_hat


class E2EModel(tf.keras.Model):
    def __init__(self, encoder, decoder, k, n, return_infobits=False, es_no=False):
        super().__init__()

        self._n = n
        self._k = k
        self._m = n - k

        self._binary_source = BinarySource()
        self._num_bits_per_symbol = 2
        self._mapper = Mapper("qam", self._num_bits_per_symbol)
        self._demapper = Demapper("app", "qam", self._num_bits_per_symbol)
        self._channel = AWGN()
        self._decoder = decoder
        self._encoder = encoder
        self._return_infobits = return_infobits
        self._es_no = es_no

    @tf.function(jit_compile=False)
    def call(self, batch_size, ebno_db):

        # no rate-adjustment for uncoded transmission or es_no scenario
        if self._decoder is not None and self._es_no==False:
            no = ebnodb2no(ebno_db, self._num_bits_per_symbol, self._k/self._n)
        else: #for uncoded transmissions the rate is 1
            no = ebnodb2no(ebno_db, self._num_bits_per_symbol, 1)

        b = self._binary_source([batch_size, self._k])
        if self._encoder is not None:
            c = self._encoder(b)
        else:
            c = b

        # check that rate calculations are correct
        assert self._n==c.shape[-1], "Invalid value of n."

        # zero padding to support odd codeword lengths
        if self._n%2==1:
            c_pad = tf.concat([c, tf.zeros([batch_size, 1])], axis=1)
        else: # no padding
            c_pad = c
        x = self._mapper(c_pad)

        y = self._channel([x, no])
        llr = self._demapper([y, no])

        # remove zero padded bit at the end
        if self._n%2==1:
            llr = llr[:,:-1]
        tf.print('PCM @ CW: ', self._decoder.pcm @
                 tf.reshape(tf.cast(c, dtype=tf.int32), (self._n, -1)) % 2)

        # decoder input nodes
        syndrome = tf.reshape( self._decoder.get_syndrome(llr),
                               (batch_size, self._m) ) # (m,n)@(n,b)->(m,b) check nodes
        x_nodes = tf.concat([llr, syndrome], axis=1)[:, :, tf.newaxis] # (b, n+m, 1)

        # and run the decoder
        if self._decoder is not None:
            tf.print('x_nodes input: ', x_nodes)
            ############################
            llr_hat = self._decoder(x_nodes)
            ############################
            # tf.print("llr_hat: ", llr_hat)

        if self._return_infobits:
            return b, llr_hat
        else:
            return c, llr_hat


# args for decoder/discriminator
args = Args(model_type='dis')
args.code.H = pcm
args.n, args.m = pcm.shape
args.k = k
args.n_steps = args.m + 5

ltd_decoder = Decoder(args) # Linear Transformer Diffusion (LTD) Decoder

e2e_ltd = E2EModel(encoder, ltd_decoder, k, n)


def bin_to_llr(x):
    """ Clip llrs to 20 for numerical stability """
    llr_vector = tf.where(x == 0, -20, 20)
    return llr_vector


def train_dec(model, args):
    # loss
    loss_fn = tf.keras.losses.MeanSquaredError()
    # optimizer
    scheduler = tf.keras.optimizers.schedules.CosineDecay(initial_learning_rate=args.lr, decay_steps=args.epochs) # 1000 is size of trainloader
    optimizer = tf.keras.optimizers.Adam(learning_rate=scheduler)
    # time start
    time_start = time.time()

    # SGD update iteration
    @tf.function(jit_compile=False)
    def train_step(batch_size):
        # train for random SNRs within a pre-defined interval
        ebno_db = tf.random.uniform([batch_size, 1],
                                    minval=args.ebno_db_min,
                                    maxval=args.ebno_db_max)

        with tf.GradientTape() as tape:
            c, llr_hat = model(batch_size, ebno_db)
            # tf.print(c, llr_hat)

            llr_y = bin_to_llr(c)
            print("llr_hat", llr_hat.shape, llr_hat.dtype)
            loss_value = loss_fn(llr_y, llr_hat)

        # and apply the SGD updates
        weights = model.trainable_weights
        grads = tape.gradient(loss_value, weights) # variables
        optimizer.apply_gradients(zip(grads, weights))
        return c, llr_hat

    print("Training Linear Transformer Diffusion Model...")
    for epoch in range(1, args.epochs + 1):
        train_step(args.batch_size)

        # eval train iter
        if epoch % args.eval_train_iter == 0:
            ebno_db = tf.random.uniform([args.batch_size, 1],
                                          minval=args.ebno_db_eval,
                                          maxval=args.ebno_db_eval)

            c, llr_hat = model(args.batch_size, ebno_db)

            # loss
            llr_y = bin_to_llr(c)
            loss_value = loss_fn(llr_y, llr_hat)

            # ber
            c_hat = llr_to_bin(llr_hat)
            ber = compute_ber(c, c_hat).numpy()

            # measure required time since last evaluation
            duration = time.time() - time_start # in s
            time_start = time.time() # reset counter

            print(f'Training epoch {epoch}/{args.epochs}, LR={optimizer.learning_rate.numpy():.2e}, Loss={loss_value.numpy():.5e}, BER={ber}, duration: {duration:.2f}s')
            break

        # save weights iter
        if epoch % args.save_weights_iter == 0:
            pass

        # heat-map visualization of the model's weights
        # for var in self.trainable_variables:
        #     var_name = var.name
        #     var_value = var.numpy()

        #     # Check if the variable is at least 2D (suitable for heatmap)
        #     if len(var_value.shape) > 1:
        #         plt.figure(figsize=(8, 6))
        #         sns.heatmap(var_value, cmap='viridis')
        #         plt.title(f'Heatmap of {var_name}')
        #         plt.show()
        #     else:
        #         print(f"{var_name} has shape {var_value.shape} which is not suitable for a heatmap.")


train_dec(e2e_ltd, args)

Training Linear Transformer Diffusion Model...
llr_hat (160, 10) <dtype: 'float32'>


llr_hat (160, 10) <dtype: 'float32'>
PCM @ CW:  [[0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 1]
 [1 0 0 ... 0 0 1]
 [1 0 0 ... 0 0 1]
 [0 1 1 ... 1 0 0]]
x_nodes input:  [[[-0.500589848]
  [7.64186668]
  [-4.37257338]
  ...
  [1]
  [0]
  [0]]

 [[-2.27644467]
  [2.71637321]
  [-2.67925882]
  ...
  [0]
  [0]
  [0]]

 [[3.25219131]
  [10.389204]
  [0.927835047]
  ...
  [1]
  [0]
  [1]]

 ...

 [[2.88789964]
  [3.67632747]
  [-3.88023782]
  ...
  [0]
  [0]
  [1]]

 [[-3.02720833]
  [-2.14785814]
  [2.68961978]
  ...
  [0]
  [1]
  [0]]

 [[-5.09194]
  [-5.47058392]
  [-0.00825977325]
  ...
  [1]
  [1]
  [0]]]
DECODER CALL
x_nodes [[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0